<a href="https://colab.research.google.com/github/IvanFXPro/Mestrado/blob/main/TestCodeClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importando las librerías necesarias y módulos
import os  # os permite interactuar con el sistema operativo
import pandas as pd  # pandas es usado para manipulación de datos
import numpy as np  # numpy es usado para cálculos numéricos
import tensorflow as tf  # tensorflow es una librería de aprendizaje automático
import matplotlib.pyplot as plt  # matplotlib se usa para visualización de datos

# Importando funcionalidades necesarias de tensorflow y keras
from tensorflow.keras.applications import ResNet50  # Modelo ResNet50 para clasificación de imágenes
from tensorflow.keras.models import Model  # Clase Model para crear modelos de redes neuronales
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D  # Diferentes capas para usar en la red neuronal
from tensorflow.keras.optimizers import Adam  # Optimizador Adam para la compilación del modelo

# Importando funcionalidades necesarias de scikit-learn
from sklearn.preprocessing import MultiLabelBinarizer  # MultiLabelBinarizer para manejar clasificación multietiqueta
from sklearn.metrics import confusion_matrix, accuracy_score, hamming_loss  # Métricas para evaluar el rendimiento del modelo
import seaborn as sns  # seaborn se usa para una mejor visualización de datos

# Importando capas adicionales para usar en el modelo
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Importando funcionalidades necesarias para la evaluación del modelo y división de datos
from sklearn.model_selection import train_test_split  # train_test_split para dividir datos en conjuntos de entrenamiento y prueba
from sklearn.metrics import classification_report, confusion_matrix  # Métricas adicionales para evaluar el modelo

# Importando el modelo VGG16 para la clasificación de imágenes
from tensorflow.keras.applications.vgg16 import VGG16

# Importando funcionalidades para la aumentación de datos
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Importando el modelo VGG19 para clasificación de imágenes
from keras.applications import VGG19

# Importando ResNet50 de nuevo (esto parece redundante y podría eliminarse)
from tensorflow.keras.applications import ResNet50

# Importando capas de nuevo (esto parece redundante y podría eliminarse)
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Importando la clase Model de nuevo (esto parece redundante y podría eliminarse)
from tensorflow.keras.models import Model

# Librería específica de Google Colab para montar Google Drive
from google.colab import drive


In [ ]:
# Montando Google Drive en el entorno de ejecución de Colab.
# Esto proporcionará acceso directo a los archivos almacenados en Google Drive.
drive.mount('/content/drive')

# Estableciendo el directorio principal donde se encuentran los datos.
# '/content/drive/My Drive/2023/LCData' es la ruta donde están almacenados los datos en Google Drive.
main_folder = '/content/drive/My Drive/2023/LCData'

# Creando una lista de subcarpetas dentro del directorio principal.
# os.listdir(main_folder) devuelve una lista de todos los archivos y carpetas en main_folder.
# os.path.isdir(os.path.join(main_folder, f)) verifica si cada elemento es un directorio.
# [f for f in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, f))] crea una lista de solo las subcarpetas.
subfolders = [f for f in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, f))]


In [ ]:
# Inicializando una lista vacía para almacenar los datos.
data = []

# Iterando sobre cada subcarpeta dentro de la lista de subcarpetas.
for subfolder in subfolders:
    # Creando la ruta completa a cada subcarpeta.
    subfolder_path = os.path.join(main_folder, subfolder)

    # Iterando sobre cada archivo dentro de las subcarpetas.
    for image_filename in os.listdir(subfolder_path):
        # Verificando si el archivo es una imagen basándonos en la extensión del archivo.
        if image_filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            # Creando la ruta completa a cada imagen.
            image_path = os.path.join(subfolder_path, image_filename)
            # Añadiendo un diccionario con el nombre de la imagen, la ruta y la etiqueta (nombre de la subcarpeta) a la lista de datos.
            data.append({'Name': image_filename, 'Path': image_path, 'Label': subfolder})

# Convirtiendo la lista de datos en un DataFrame de pandas para una manipulación más fácil de los datos.
df_total = pd.DataFrame(data)

# Ajustando las opciones de visualización de pandas para mostrar todas las columnas.
pd.set_option('display.max_columns', None)

# Imprimiendo el DataFrame para visualizar los datos.
print(df_total)

In [ ]:

# Asignando df_total a original_df para mantener una referencia del DataFrame original
# y poder trabajar con una copia sin modificar el DataFrame original.
original_df = df_total

# Extrayendo todas las etiquetas (columna 'Label') de original_df para su análisis o uso posterior.
labels = original_df['Label']

print('---------------------------------------------------------------------------------------------')

# Calculando y mostrando la frecuencia de cada etiqueta única en el DataFrame.
# Es útil para entender la distribución de clases y cuántas muestras hay de cada clase en el conjunto de datos.
label_counts = original_df['Label'].value_counts()
print(label_counts)

# Identificando la clase con la menor cantidad de muestras en el conjunto de datos.
# Esto puede ser útil para identificar si alguna clase está subrepresentada y puede necesitar oversampling.
min_class_name, min_count = min(label_counts.items(), key=lambda x: x[1])
print("CLASS " + str(min_class_name) + " appears " + str(min_count) + " Times ")

print('----------------------------------------------------------------------------------------------')

In [ ]:
# Creando un diccionario que contiene el número deseado de muestras para cada clase/etiqueta.
# Cada clave del diccionario representa una clase/etiqueta (por ejemplo, "Malignant", "Normal", "Benign").
# Cada valor asociado representa el número de muestras deseadas para esa clase específica.
desired_samples_per_class = {
    "Malignant": 500,  # Se desean 500 muestras para la clase "Malignant".
    "Normal": 400,     # Se desean 400 muestras para la clase "Normal".
    "Bengin": 100,     # Se desean 100 muestras para la clase "Benign".
}


In [ ]:
# Creando un nuevo DataFrame vacío llamado balanced_df.
# Este DataFrame se utilizará posteriormente para almacenar las muestras de cada clase/etiqueta,
# asegurando que el conjunto de datos esté balanceado según el número de muestras deseadas
# definidas previamente en desired_samples_per_class.
balanced_df = pd.DataFrame()


In [ ]:
# Iterando sobre cada etiqueta/clase y el número de muestras deseadas de desired_samples_per_class.
for label, num_samples in desired_samples_per_class.items():
    # Seleccionando un subconjunto del DataFrame original que corresponde a la etiqueta/clase actual.
    class_subset = original_df[original_df['Label'] == label]

    # Muestreando aleatoriamente el número deseado de muestras de cada clase.
    # 'random_state=42' asegura que la aleatoriedad sea reproducible y obtengamos los mismos resultados en cada ejecución.
    sampled_subset = class_subset.sample(n=num_samples, random_state=42)

    # Concatenando el subconjunto muestreado al DataFrame balanceado.
    # 'ignore_index=False' mantiene los índices originales del DataFrame. Si se prefiere reiniciar los índices, se puede cambiar a True.
    balanced_df = pd.concat([balanced_df, sampled_subset], ignore_index=False)

In [ ]:
# Extrayendo las etiquetas del DataFrame balanceado para su posterior análisis o uso.
labels = balanced_df['Label']

# Contando la cantidad de ocurrencias de cada etiqueta única en el DataFrame balanceado
# Esto es útil para verificar si el DataFrame ha sido balanceado correctamente.
label_counts = balanced_df['Label'].value_counts()

# Imprimiendo las cuentas de etiquetas para visualizar la distribución de las muestras entre diferentes clases/etiquetas.
print(label_counts)


In [ ]:
# Identificando los índices que están presentes en el DataFrame original pero no en el DataFrame balanceado.
# Esto se hace para encontrar las muestras que no se han utilizado durante el balanceo.
unused_indices = original_df.index.difference(balanced_df.index)

# Creando un nuevo DataFrame, mytest_df, que contiene solo las muestras no utilizadas.
# Estas muestras se pueden usar para testing u otros propósitos, ya que no están presentes en el DataFrame balanceado.
mytest_df = original_df.loc[unused_indices]

# Contando la cantidad de ocurrencias de cada etiqueta única en el nuevo DataFrame (mytest_df).
# Esto es útil para entender la distribución de las etiquetas en el conjunto de datos no utilizado.
label_counts = mytest_df['Label'].value_counts()

# Imprimiendo las cuentas de etiquetas para visualizar la cantidad de muestras no utilizadas de cada clase/etiqueta.
print(label_counts)


In [ ]:
# Restableciendo los índices del DataFrame balanced_df.
# El parámetro drop=True descarta la columna de índice actual en lugar de mantenerla como una columna separada.
# El parámetro inplace=True modifica el DataFrame original y no devuelve un nuevo objeto DataFrame.
balanced_df.reset_index(drop=True, inplace=True)

# Realizando el mismo proceso de restablecimiento de índices para el DataFrame mytest_df.
# Esto asegura que ambos DataFrames tengan índices limpios y consecutivos después de las operaciones previas.
mytest_df.reset_index(drop=True, inplace=True)

In [ ]:
# Dividiendo el DataFrame balanceado en conjuntos de entrenamiento y validación.
# test_size=0.2: El 20% de los datos se reservan para el conjunto de validación, y el 80% restante se usa para entrenamiento.
# random_state=42: Asegura que la división sea reproducible al usar una semilla específica para el generador de números aleatorios.
# stratify=balanced_df['Label']: Asegura que la distribución de las etiquetas/clases sea similar tanto en los conjuntos de entrenamiento como de validación,
#                                manteniendo las proporciones de las diferentes clases.
train_df2, valid_df2 = train_test_split(balanced_df, test_size=0.2, random_state=42, stratify=balanced_df['Label'])

In [ ]:
# Definir el tamaño de las imágenes que se usarán en el modelo.
# Esto es importante para asegurarse de que todas las imágenes tengan las mismas dimensiones.
image_size = (224, 224)  # Adjust this size as needed

# Definir el tamaño del lote o 'batch'.
# Esto se refiere al número de ejemplos de entrenamiento utilizados en cada iteración para actualizar los pesos del modelo.
batch_size = 32

# Crear un generador de imágenes. Esto ayudará a preprocesar las imágenes antes de alimentarlas al modelo.
# En este caso, las imágenes se están normalizando, dividiéndolas por 255 para que los píxeles estén entre 0 y 1.
datagen_level2 = ImageDataGenerator(rescale=1.0/255.0)  # Normalize pixel values

# Configurar el generador de datos de entrenamiento.
# Este generador tomará un DataFrame como input y generará lotes de imágenes preprocesadas y etiquetas correspondientes.
train_generator_level2 = datagen_level2.flow_from_dataframe(
    train_df2,  # DataFrame que contiene los datos de entrenamiento
    x_col="Path",  # Columna en el DataFrame que contiene la ruta de las imágenes
    y_col="Label",  # Columna en el DataFrame que contiene las etiquetas de las imágenes
    target_size=image_size,  # Tamaño al cual se redimensionarán las imágenes
    batch_size=batch_size,  # Número de ejemplos en cada lote
    class_mode="categorical"  # Modo de las etiquetas. En este caso, las etiquetas se consideran categóricas (más de 2 clases)
)

# Configurar el generador de datos de validación de manera similar al generador de datos de entrenamiento.
validation_generator_level2 = datagen_level2.flow_from_dataframe(
    valid_df2,
    x_col="Path",
    y_col="Label",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical"
)

# Configurar el generador de datos de prueba. Esto se utilizará para evaluar el modelo después del entrenamiento.
testsubset_df=mytest_df
test_generator_level2 = datagen_level2.flow_from_dataframe(
    testsubset_df,
    x_col="Path",
    y_col="Label",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical"
)

In [ ]:
# Aquí estamos obteniendo las etiquetas de las clases del generador de entrenamiento.
# Multiclass_labels almacenará un diccionario donde las claves son los nombres de las clases (etiquetas)
# y los valores son los índices numéricos correspondientes asignados a cada clase única durante el flujo de datos.
# Esto es útil para tener una referencia de qué índice numérico corresponde a cada clase/etiqueta cuando estamos
# trabajando con un problema de clasificación multiclase.
Multiclass_labels = train_generator_level2.class_indices

In [ ]:
# Aquí se inicializa el modelo base usando ResNet50 preentrenado en ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# A continuación, construimos la parte superior personalizada del modelo
x = base_model.output  # Obtiene la salida del modelo base
x = GlobalAveragePooling2D()(x)  # Añade una capa de agrupamiento global promedio
x = Dense(128, activation='relu')(x)  # Añade una capa densa con 128 nodos y activación relu
# Aquí, definimos la capa de salida con tantos nodos como etiquetas y activación softmax
predictions_level2 = Dense(len(Multiclass_labels), activation='softmax')(x)
# Ahora, combinamos el modelo base y las capas superiores personalizadas en un modelo completo
multiclass_model1 = Model(inputs=base_model.input, outputs=predictions_level2)

# En esta parte, hacemos que las capas del modelo base sean no entrenables
for layer in base_model.layers:
    layer.trainable = False

# Compilamos el modelo, especificando el optimizador, la función de pérdida y las métricas para monitorear
multiclass_model1.compile(optimizer=Adam(learning_rate=0.001),
                         loss='categorical_crossentropy',
                         metrics=['accuracy'])

# Finalmente, ajustamos el modelo a los datos de entrenamiento y validación, especificando el número de épocas
history1 = multiclass_model1.fit(train_generator_level2,
                                validation_data=validation_generator_level2,
                                epochs=10)

In [ ]:
# Utilizando VGG16 como modelo base preentrenado
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Añadiendo capas personalizadas para adaptar la salida del modelo base a nuestras necesidades
x = base_model.output
x=MaxPooling2D(pool_size=(3, 3))(x)  # Añadiendo una capa de MaxPooling
x = GlobalAveragePooling2D()(x)  # Añadiendo una capa de GlobalAveragePooling
x = Dense(128, activation='relu')(x)  # Añadiendo una capa densa con activación relu
# Añadiendo una capa de salida con tantas unidades como clases y activación softmax
predictions_level2 = Dense(len(Multiclass_labels), activation='softmax')(x)

# Creando el modelo final que será entrenado
multiclass_model2 = Model(inputs=base_model.input, outputs=predictions_level2)

In [ ]:
# En este segmento de código, estamos configurando las capas del modelo base para que no sean entrenables.
# Esto significa que los pesos de estas capas no se actualizarán durante el entrenamiento.
for layer in base_model.layers:
    layer.trainable = False

# A continuación, compilamos el modelo. Esto prepara el modelo para el entrenamiento.
# Se especifica el optimizador 'adam', la función de pérdida 'categorical_crossentropy' y
# la métrica 'accuracy' que queremos monitorear.
multiclass_model2.compile(optimizer='adam',
                         loss='categorical_crossentropy',
                         metrics=['accuracy'])

# Aquí, estamos ajustando el modelo a los datos. Específicamente, estamos entrenando el modelo
# usando un generador de datos, lo que significa que los datos se generarán y alimentarán al modelo
# en tiempo real. También estamos especificando datos de validación y el número de épocas
# que queremos que se ejecute el entrenamiento.
history2 = multiclass_model2.fit(train_generator_level2,
                                validation_data=validation_generator_level2,
                                epochs=10)


In [ ]:
# Utilizando VGG19 como modelo base preentrenado
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Añadiendo capas personalizadas para adaptar la salida del modelo base a nuestras necesidades
x = base_model.output
x=MaxPooling2D(pool_size=(2, 2))(x)  # Añadiendo una capa de MaxPooling
x = GlobalAveragePooling2D()(x)  # Añadiendo una capa de GlobalAveragePooling

x = Dense(128, activation='relu')(x)  # Añadiendo una capa densa con activación relu
# Añadiendo una capa de salida con tantas unidades como clases y activación softmax
predictions_level2 = Dense(len(Multiclass_labels), activation='softmax')(x)

# Creando el modelo final que será entrenado
multiclass_model3 = Model(inputs=base_model.input, outputs=predictions_level2)

# Asegurando que las capas del modelo base no serán entrenadas
for layer in base_model.layers:
    layer.trainable = False

# Compilando el modelo con el optimizador adam, y configurándolo para clasificación multiclase
multiclass_model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Iniciando el entrenamiento del modelo utilizando los datos generados y validándolo también.
history3 = multiclass_model3.fit(train_generator_level2, validation_data=validation_generator_level2, epochs=10)

In [ ]:
# Creando una figura para visualizar la precisión y la pérdida durante el entrenamiento
plt.figure(figsize=(10, 4))

# Subgráfico para la precisión durante el entrenamiento
plt.subplot(1, 2, 1)
# Graficando la precisión de entrenamiento y validación del modelo con ResNet
plt.plot(history1.history['accuracy'], label='Training Accuracy Resnet')
plt.plot(history1.history['val_accuracy'], label='Validation Accuracy Resnet')
# Graficando la precisión de entrenamiento y validación del modelo con VGG16
plt.plot(history2.history['accuracy'], label='Training Accuracy VGG16')
plt.plot(history2.history['val_accuracy'], label='Validation Accuracy VGG16')
# Las líneas para VGG19 están comentadas, se pueden descomentar si se desea visualizar
#plt.plot(history3.history['accuracy'], label='Training Accuracy VGG19')
#plt.plot(history3.history['val_accuracy'], label='Validation Accuracy VGG19')

plt.xlabel('Epoch')  # Etiqueta del eje x
plt.ylabel('Accuracy')  # Etiqueta del eje y
plt.legend()  # Añadiendo la leyenda para identificar cada línea

# Subgráfico para la pérdida durante el entrenamiento
plt.subplot(1, 2, 2)
# Graficando la pérdida de entrenamiento y validación del modelo con ResNet
plt.plot(history1.history['loss'], label='Training Loss Resnet')
plt.plot(history1.history['val_loss'], label='Validation Loss Resnet')
# Graficando la pérdida de entrenamiento y validación del modelo con VGG16
plt.plot(history2.history['loss'], label='Training Loss VGG16')
plt.plot(history2.history['val_loss'], label='Validation Loss VGG16')
# Las líneas para VGG19 están comentadas, se pueden descomentar si se desea visualizar
#plt.plot(history3.history['loss'], label='Training Loss VGG19')
#plt.plot(history3.history['val_loss'], label='Validation Loss VGG19')

plt.xlabel('Epoch')  # Etiqueta del eje x
plt.ylabel('Loss')  # Etiqueta del eje y
plt.legend()  # Añadiendo la leyenda para identificar cada línea

plt.show()  # Mostrando la figura

In [ ]:
# Evaluando el modelo multiclase basado en ResNet en el conjunto de datos de prueba
loss1, accuracy1 = multiclass_model1.evaluate(test_generator_level2)

# Imprimiendo la pérdida y la precisión del modelo
print(f"Test Loss for Resnet: {loss1:.4f}")  # Imprime la pérdida con 4 decimales
print(f"Test Accuracy for Resnet: {accuracy1 * 100:.2f}%")  # Imprime la precisión como un porcentaje con 2 decimales

In [ ]:
true_labels = []  # Inicialización de la lista para las etiquetas verdaderas
predicted_labels = []  # Inicialización de la lista para las etiquetas predichas
from tensorflow.keras.preprocessing import image  # Importando el módulo necesario para procesamiento de imágenes

In [ ]:
# Definición de una función para preprocesar las imágenes antes de alimentarlas al modelo.
# La función toma la ruta de una imagen como entrada.
def preprocess_image(image_path):
    # Cargando la imagen de la ruta especificada y ajustando su tamaño a 224x224 píxeles.
    # 224x224 es una dimensión comúnmente usada para modelos preentrenados.
    img = image.load_img(image_path, target_size=(224, 224))

    # Convirtiendo la imagen cargada en un array de numpy para poder manipularla y procesarla.
    img_array = image.img_to_array(img)

    # Añadiendo una dimensión adicional que representa el tamaño del batch.
    # Esto es necesario porque muchos modelos de Keras esperan datos con una dimensión de batch.
    img_array = np.expand_dims(img_array, axis=0)

    # Normalizando los valores de los píxeles de la imagen para que estén entre 0 y 1.
    # Esto es una práctica común que ayuda a mejorar la convergencia durante el entrenamiento.
    img_array = img_array / 255.0

    # Retornando el array de la imagen preprocesada.
    return img_array


# Inicializando dos listas vacías para almacenar las etiquetas verdaderas y las etiquetas predichas
# de las imágenes cuando sean procesadas y evaluadas por un modelo.
true_labels = []        # Esta lista almacenará las etiquetas verdaderas/reales de las imágenes.
predicted_labels = []   # Esta lista almacenará las etiquetas que el modelo prediga.

In [ ]:
# Iterando sobre cada fila en el DataFrame testsubset_df, que asumimos contiene información sobre imágenes y sus etiquetas.
for index, row in testsubset_df.iterrows():
    # Obteniendo la ruta de la imagen y la etiqueta de la fila actual del DataFrame.
    filename = row['Path']
    label = row['Label']

    # Utilizando la función de preprocesamiento definida previamente para preparar la imagen para la predicción.
    img_array = preprocess_image(filename)

    # Realizando predicciones con el modelo multiclase preentrenado.
    # Las predicciones resultantes son probabilidades para cada clase.
    multi_class_predictions = multiclass_model1.predict(img_array)

    # Identificando el índice de la clase con la mayor probabilidad predicha.
    sub_class_index = np.argmax(multi_class_predictions)

    # Buscando el nombre de la clase correspondiente al índice identificado.
    sub_class_name = [k for k, v in train_generator_level2.class_indices.items() if v == sub_class_index][0]

    # Imprimiendo el nombre de la clase detectada y el índice actual de la iteración.
    print(f"Detected-class: " + sub_class_name)
    print(index)

    # Añadiendo el nombre de la clase predicha a la lista de etiquetas predichas.
    predicted_labels.append(sub_class_name)

    # Añadiendo la etiqueta real de la imagen a la lista de etiquetas verdaderas.
    true_labels.append(label)

In [ ]:
# Generando una matriz de confusión usando las etiquetas verdaderas y las etiquetas predichas.
# Esta matriz ayuda a entender cómo se distribuyen las predicciones reales vs. las predicciones del modelo.
confusion_matrix_subclasses_abnormal = confusion_matrix(true_labels, predicted_labels)

# Calculando la precisión integrada multi-clase.
# Es el porcentaje de predicciones correctas con respecto a todas las predicciones.
IntegratedMultiAcc = (np.trace(confusion_matrix_subclasses_abnormal) / np.sum(confusion_matrix_subclasses_abnormal)) * 100

# Extrayendo los nombres de las etiquetas de clases del generador de entrenamiento.
# Esto es útil para etiquetar los ejes en la visualización de la matriz de confusión.
class_labels = [label for label in train_generator_level2.class_indices]

# Visualizando la matriz de confusión usando seaborn.
plt.figure(figsize=(8, 6))  # Estableciendo el tamaño de la figura.

# Dibujando un mapa de calor (heatmap) para representar la matriz de confusión.
# Los números reales (annot=True) se muestran en cada celda, y se utiliza una paleta de colores azules.
sns.heatmap(confusion_matrix_subclasses_abnormal, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels, yticklabels=class_labels)

# Etiquetando los ejes y dando un título al gráfico.
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Resnet Multi_Class Inside Abnormal Accuracy=' + str(IntegratedMultiAcc))

# Mostrando el gráfico.
plt.show()

In [ ]:
# Evaluando el rendimiento del modelo multiclass_model2 en el conjunto de prueba.
# La función 'evaluate' devuelve la pérdida y la precisión del modelo en el conjunto de datos proporcionado.
loss2, accuracy2 = multiclass_model2.evaluate(test_generator_level2)

# Imprimiendo la pérdida obtenida en el conjunto de prueba para el modelo VGG16.
print(f"Test Loss for VGG16: {loss2:.4f}")

# Imprimiendo la precisión obtenida en el conjunto de prueba para el modelo VGG16.
# La precisión se multiplica por 100 para convertirla en un porcentaje.
print(f"Test Accuracy for VGG16: {accuracy2 * 100:.2f}%")

In [ ]:
# Inicializando dos listas vacías para almacenar las etiquetas verdaderas y las etiquetas predichas.
true_labels = []
predicted_labels = []

# Importando la función 'image' de tensorflow.keras.preprocessing para cargar y preprocesar imágenes.
from tensorflow.keras.preprocessing import image

# Definiendo una función para preprocesar las imágenes antes de alimentarlas al modelo para la predicción.
def preprocess_image(image_path):
    # Cargando la imagen del path especificado y ajustando su tamaño a 224x224 píxeles.
    img = image.load_img(image_path, target_size=(224, 224))

    # Convirtiendo la imagen cargada en un array de numpy.
    img_array = image.img_to_array(img)

    # Expandiendo las dimensiones del array de la imagen, añadiendo una dimensión para el batch size.
    img_array = np.expand_dims(img_array, axis=0)

    # Normalizando los valores de los píxeles de la imagen para que estén entre 0 y 1.
    img_array = img_array / 255.0

    return img_array  # Retornando el array de la imagen preprocesada.

# Reinicializando las listas de etiquetas verdaderas y etiquetas predichas (esto podría ser necesario si
# desea reutilizar las listas y asegurarse de que estén vacías al principio).
true_labels = []
predicted_labels = []

In [ ]:
# Iterando a través de cada fila en el dataframe testsubset_df.
for index, row in testsubset_df.iterrows():
    # Obteniendo el nombre del archivo (path completo) y la etiqueta de la imagen actual.
    filename = row['Path']
    label = row['Label']

    # Preprocesando la imagen usando la función definida anteriormente.
    img_array = preprocess_image(filename)

    # Realizando la predicción utilizando el modelo multiclase.
    # Esto nos da una lista de probabilidades para cada clase.
    multi_class_predictions = multiclass_model2.predict(img_array)

    # Identificando el índice de la clase con la probabilidad más alta en las predicciones.
    sub_class_index = np.argmax(multi_class_predictions)

    # Buscando el nombre de la clase que corresponde al índice identificado.
    sub_class_name = [k for k, v in train_generator_level2.class_indices.items() if v == sub_class_index][0]

    # Imprimiendo la clase detectada y el índice actual en el dataframe.
    print(f"Detected-class: " + sub_class_name)
    print(index)

    # Añadiendo la clase predicha a la lista de etiquetas predichas.
    predicted_labels.append(sub_class_name)

    # Añadiendo la etiqueta verdadera a la lista de etiquetas verdaderas.
    true_labels.append(label)

In [ ]:
# Calculando la matriz de confusión utilizando las etiquetas verdaderas y las etiquetas predichas.
confusion_matrix_subclasses_abnormal = confusion_matrix(true_labels, predicted_labels)

# Calculando la precisión integrada multi-clase como el porcentaje de predicciones correctas.
IntegratedMultiAcc = (np.trace(confusion_matrix_subclasses_abnormal) / np.sum(confusion_matrix_subclasses_abnormal)) * 100

# Obteniendo las etiquetas de las clases del generador.
class_labels = [label for label in train_generator_level2.class_indices]

# Creando una figura para visualizar la matriz de confusión.
plt.figure(figsize=(8, 6))

# Creando un mapa de calor para la matriz de confusión usando seaborn.
# Anotaciones se habilitan y el formato se establece como decimal.
sns.heatmap(confusion_matrix_subclasses_abnormal, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels, yticklabels=class_labels)

# Etiquetando los ejes x e y del mapa de calor.
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')

# Añadiendo un título a la figura que muestra la precisión integrada multi-clase.
plt.title('VGG16-Multi_Class Inside Abnormal Accuracy=' + str(IntegratedMultiAcc))

# Mostrando la figura.
plt.show()

In [ ]:
# Evaluando el modelo multiclase (multiclass_model3) usando el conjunto de datos de prueba (test_generator_level2).
# Esto devolverá la pérdida y la precisión del modelo en el conjunto de datos de prueba.
loss3, accuracy3 = multiclass_model3.evaluate(test_generator_level2)

# Imprimiendo la pérdida y la precisión del modelo en el conjunto de datos de prueba.
# La pérdida se muestra con 4 decimales y la precisión se muestra como un porcentaje con 2 decimales.
print(f"Test Loss for VGG19: {loss3:.4f}")
print(f"Test Accuracy for VGG19: {accuracy3 * 100:.2f}%")

In [ ]:
# Inicializando listas vacías para almacenar las etiquetas verdaderas y las etiquetas predichas.
true_labels = []
predicted_labels = []

# Importando la función image del módulo de preprocesamiento de keras.
from tensorflow.keras.preprocessing import image

# Definiendo una función para preprocesar las imágenes.
def preprocess_image(image_path):
    # Cargando la imagen del path especificado y ajustándola al tamaño 224x224.
    img = image.load_img(image_path, target_size=(224, 224))

    # Convirtiendo la imagen cargada en un array.
    img_array = image.img_to_array(img)

    # Expandiendo las dimensiones del array de la imagen, añadiendo una dimensión para el batch.
    img_array = np.expand_dims(img_array, axis=0)  # Añadiendo dimensión de batch

    # Normalizando los valores de los píxeles de la imagen para que estén entre 0 y 1.
    img_array = img_array / 255.0  # Normalizando valores de píxeles

    return img_array  # Retornando el array de la imagen preprocesada.

# Re-inicializando las listas de etiquetas verdaderas y predichas, asegurándose de que estén vacías.
true_labels = []
predicted_labels = []

In [ ]:
# Iterando sobre cada fila en el dataframe testsubset_df.
for index, row in testsubset_df.iterrows():
        # Obteniendo el nombre del archivo (path) y la etiqueta real de la imagen.
        filename = row['Path']
        label = row['Label']

        # Preprocesando la imagen usando la función previamente definida.
        img_array = preprocess_image(filename)

        # Obteniendo las predicciones del modelo para la imagen preprocesada.
        multi_class_predictions = multiclass_model3.predict(img_array)

        # Identificando el índice de la clase con la mayor probabilidad predicha.
        sub_class_index = np.argmax(multi_class_predictions)  # Elegir la sub-clase con la mayor probabilidad

        # Obteniendo el nombre de la clase correspondiente al índice identificado.
        sub_class_name = [k for k, v in train_generator_level2.class_indices.items() if v == sub_class_index][0]

        # Imprimiendo la clase detectada y el índice de la fila actual.
        print(f"Detected-class: " +sub_class_name)  # Añadir 1 al índice para coincidir con sus etiquetas de clase
        print(index)

        # Añadiendo las etiquetas predichas y reales a las listas correspondientes.
        predicted_labels.append(sub_class_name)
        true_labels.append(label)

In [ ]:
# Calculando la matriz de confusión entre las etiquetas verdaderas y las predicciones
confusion_matrix_subclasses_abnormal = confusion_matrix(true_labels, predicted_labels)

# Calculando la precisión integrada multi-clase, que es la suma de la diagonal (verdaderos positivos)
# de la matriz de confusión dividida por el total de predicciones.
IntegratedMultiAcc = (np.trace(confusion_matrix_subclasses_abnormal) / np.sum(confusion_matrix_subclasses_abnormal)) * 100

# Obteniendo las etiquetas de las clases del generador de datos para entrenamiento.
class_labels = [label for label in train_generator_level2.class_indices]

# Creando una figura para visualizar la matriz de confusión
plt.figure(figsize=(8, 6))

# Visualizando la matriz de confusión como un mapa de calor usando seaborn
sns.heatmap(confusion_matrix_subclasses_abnormal, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)

# Etiquetando los ejes x e y
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')

# Titulando la visualización con la precisión calculada anteriormente
plt.title('VGG19-Multi_Class Inside Abnormal Accuracy=' + str(IntegratedMultiAcc))

# Mostrando la visualización
plt.show()
